# CVXPY

## 발전량 최적화 (LP & QP)

- 간단한 LP 및 QP 구현 예시
- 작성자 : 차지원 (ryunx21@gmail.com)
- 발표자료 : https://www.slideshare.net/JiwonCha3/pyconkr-2019-164237844

![ex_screenshot](../image/example.png)

In [1]:
import cvxpy as cp
import numpy as np

# 현재 사용 가능한 솔버 확인
cp.installed_solvers()

['CVXOPT', 'ECOS', 'ECOS_BB', 'GLPK', 'GLPK_MI', 'OSQP', 'SCS']

In [2]:
# 변수 선언, p는 0 또는 양수
p = cp.Variable(3, nonneg=True)

# 제약조건
constraints = list()

# 설비제약 (elemental wise)
constraints.append(p[0] >= 500)
constraints.append(p[0] <= 1000)
constraints.append(p[1] >= 300)
constraints.append(p[1] <= 500)
constraints.append(p[2] >= 0)
constraints.append(p[2] <= 200)

# 수급제약
# 등식 제약으로 넣어도 되지만, 목적함수에서 p를 최소화하기 때문에
# 수렴성을 위해 relaxation 했음
constraints.append(cp.sum(p) >= 1400)

# 목적함수
obj = cp.Minimize(20*p[0] + 40*p[1] + 10*p[2])

prob = cp.Problem(obj, constraints)

In [3]:
prob.solve(solver='GLPK')
print(p.value)
# [900. 300. 200.]

[900. 300. 200.]


In [4]:
prob.solve(solver='OSQP')
print(p.value)
# [900. 300. 200.]

[900. 300. 200.]


In [5]:
prob.solve(solver='CVXOPT')
print(p.value)
#[899.9999985  299.99999262 200.00000273]

[899.9999985  299.99999262 200.00000273]


In [6]:
prob.solve(solver='ECOS')
print(p.value)
# [899.99999998 299.9999999  200.00000004]

[899.99999998 299.9999999  200.00000004]


# CVXPY expr 사용을 통한 수식 간략화

In [7]:
# 변수 선언, p는 0 또는 양수
p = cp.Variable(3,nonneg=True)

# 제약조건 리스트
constraints = list()

# 수급제약
constraints.append(
    cp.sum(p) >= 1400)

# 설비제약 (간단하게)
constraints.append(p>=[500,300,0])
constraints.append(p<=[1000,500,200])

# 목적함수(QP) (C1:1차, C2:2차계수)
c1 = np.array([20,40,10])
c2 = np.array([0.1,0.04,0.01])
obj = cp.Minimize(
    cp.sum(p*c1)
    +cp.sum(cp.sum_squares(p)*c2))

prob = cp.Problem(obj, constraints)

In [8]:
prob.solve(solver='OSQP')
print(p.value)
# [700. 500. 200.]

[700. 500. 200.]
